## Amazon S3

### What it is S3

__Amazon S3__ (Simple Storage Service) is a Amazon's service for storing files. It is simple in a sense that one store data using the follwing:
* __bucket__: place to store. Its name is unique for all S3 users, which means that there cannot exist two buckets with the same name even if they are private for to different users.
* __key__: a unique (for a bucket) name that link to the sotred object. It is common to use path like syntax to group objects. 
* __object__: any file (text or binary). It can be partitioned.

### Sign up
First go to 
<https://s3.console.aws.amazon.com/s3>

and sign up for S3. You can also try to create a bucket, upload files etc. Here we will explain how to use it porogramatically. 

## Data 

But first let's get data we are going to use here. We take the dataset `train.csv` from <https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge>. 
We locally store in `data` directory.

### Sampling data

We also sample this dataset in order to have one more example (and faster execution).

In [ ]:
import numpy as np
import pandas as pd
np.random.seed(10)
comments = pd.read_csv("data/train.csv")
nrows = comments.shape[0]
comments.iloc[np.random.choice(range(nrows), 10000, replace=False)]\
    .to_csv("data/train_sample10000.csv", index=False)
comments.iloc[np.random.choice(range(nrows), 1000, replace=False)]\
    .to_csv("data/train_sample1000.csv", index=False)
comments.iloc[np.random.choice(range(nrows), 100, replace=False)]\
    .to_csv("data/train_sample100.csv", index=False)
comments10 = comments.iloc[np.random.choice(range(nrows), 10, replace=False)]
comments10.to_csv("data/train_sample10.csv", index=False)
comments10

## Installing AWS Command Line Interface and boto

In order to install boto (Python interface to Amazon Web Service) and AWS Command Line Interface (__CLI__) type:
```
pip install boto3
pip install awscli
```

Then in your home directory create file `~/.aws/credentials` with the following:

```
[myaws]
aws_access_key_id = YOUR_ACCESS_KEY
aws_secret_access_key = YOUR_SECRET_KEY
```

If you add these configuration as `[default]`, you won't need to add `--profile myaws` in CLI commands in Section CLI Basic Commands.

### Where to get credentials from

1. Go to https://console.aws.amazon.com/console/home and log in
2. Click on USER NAME (right top) and select `My Security Credentials`.
3. Click on `+ Access keys (access key ID and secret access key)` and then on `Create New Acess Key`.
4 Choose `Show access key`.

## CLI Basic Commands 

### List buckets
```
aws --profile myaws s3 ls
```

### List all buckets

```
aws --profile myaws s3 ls 
```

### Create buckers
```
aws --profile myaws s3 mb s3://barteks-toxic-comments
```
__Warning__ The bucket namespace is shared by all users of the system so you need to change the name.

### Upload and download files

#### Upload
```
aws --profile myaws s3 cp data/train.csv s3://barteks-toxic-comments
aws --profile myaws s3 cp data/train_sample10000.csv s3://barteks-toxic-comments/sample/
aws --profile myaws s3 cp data/train_sample1000.csv s3://barteks-toxic-comments/sample/
aws --profile myaws s3 cp data/train_sample100.csv s3://barteks-toxic-comments/sample/
aws --profile myaws s3 cp data/train_sample10.csv s3://barteks-toxic-comments/sample/
```

The last 4 commands can be done in shell calling:
```
for f in data/train_sample1*.csv; do aws --profile myaws s3 cp $f s3://barteks-toxic-comments/sample/; done
```

#### Download
```
aws --profile myaws s3 cp s3://barteks-toxic-comments/sample/train_sample10.csv data/train_copy_sample10.csv
```

### List files in path
 
```
aws --profile myaws s3 ls s3://barteks-toxic-comments/
aws --profile myaws s3 ls s3://barteks-toxic-comments/sample/
```

### Remove file(s)

```
aws --profile myaws s3 rm s3://barteks-toxic-comments/sample/train_sample2.csv
aws --profile myaws s3 rm s3://barteks-toxic-comments/sample/ --recursive
```

### Delete bucket

For deleting a bucket use
```
aws --profile myaws s3 rb  s3://barteks-toxic-comments
```
in order to delete non empty backet use `--force` option.

In order to empty a backet use
```
aws --profile myaws s3 rm s3://barteks-toxic-comments/ --recursive
```

## What Boto is

Boto is a Python package that provides interfaces to Amazon Web Services. Here we are focused on its application to S3.

### Creating S3 Resource

We start using boto3 by creating S3 resorce object.

In [ ]:
import boto3
session = boto3.Session(profile_name='myaws')
s3 = session.resource('s3')

#### From evironment variables

If your credentials are stored as evirionment variables `AWS_SECRET_KEY_ID` and `AWS_SECRET_ACCESS_KEY` then you can do the following:

```
import os
aws_access_key_id = os.environ.get('AWS_SECRET_KEY_ID')
aws_secret_access_key = s.environ.get('AWS_SECRET_ACCESS_KEY')
session = boto3.Session(
    aws_access_key_id=aws_access_key_id, 
    aws_secret_access_key=aws_secret_access_key)
```

### List buckets

In [ ]:
list(s3.buckets.all())

### Create a bucket

__Warning__ As before, bucket's namespace is shared, so the following command may not poroduce a bucket if a bucket with the name exists.

In [ ]:
#s3.create_bucket(
#    ACL='public-read',
#    Bucket="barteks-toxic-comments-stats")

And you have the followng Access Control List (ACL) options while creating it: 
* `'private', 
* 'public-read', 
* 'public-read-write', 
* 'authenticated-read'`.

### Deleting

In [ ]:
#bucket = s3.Bucket('barteks-toxic-comments-stats')
#bucket.delete()

### List keys in the bucket

In [ ]:
bucket = s3.Bucket('barteks-toxic-comments')
objs = [obj for obj in bucket.objects.all()]
objs

In [ ]:
[obj.key for obj in bucket.objects.filter(Prefix="sample/")]

The object of class `ObjectSummary` has to properties `Bucket` (that returns Bucket object), `bucket_name` and `key` that return strings. 

In [ ]:
objs[0].Bucket(), objs[0].bucket_name, objs[0].key

#### Filter keys and sort them 

In [ ]:
objects = [obj for obj in bucket.objects.filter(Prefix="sample/")]
objects.sort(key=lambda obj: obj.key, reverse=True)
objects

### Download file

In [ ]:
bucket = s3.Bucket('barteks-toxic-comments')
bucket.download_file('sample/train_sample10.csv', "data/train_copy2_sample10.csv")

#### Transform to pandas.DataFrame

One way to do this is to download the file and open it with `pandas.read_csv` method. If we do not want to do this we have to read it a buffer and open it from there. In order to do this we need to use low level interaction.

In [ ]:
import io
obj = s3.Object('barteks-toxic-comments', 'sample/train_sample100.csv').get()
comments100 = pd.read_csv(io.BytesIO(obj['Body'].read()))
comments100.head()

Another way, using higher level `download_fileobj` requires transform bytes streaiming into text streaming.

In [ ]:
f = io.BytesIO()
bucket.download_fileobj('sample/train_sample10.csv', f)
f.seek(0)
pd.read_csv(io.TextIOWrapper(f, encoding='utf-8'))

### Upload file

In [ ]:
stat_bucket = s3.Bucket("barteks-toxic-comments-stats")

In [ ]:
comments100stat = \
    comments100.groupby(["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"])\
    .count().reset_index()
comments100stat.to_csv("data/train_sample100stat.csv", index=False)

In [ ]:
stat_bucket.upload_file("data/train_sample100stat.csv", 'sample/train_sample100stat.csv')

In [ ]:
list(bucket.objects.all())

#### With buffer

In [ ]:
import io
f = io.StringIO()
comments100stat.to_csv(f, index=False)
stat_bucket.upload_fileobj(f, 'sample/train_sample100stat_copy.csv')

In [ ]:
list(bucket.objects.all())

### Delete

In [ ]:
obj = s3.Object('barteks-toxic-comments', 'sample/train_copy2_sample10.csv')

In [ ]:
obj.delete()

### S3 client: low level access

In [ ]:
s3_client = session.client('s3')

## Access through http(s)

### Change Access Control

In [ ]:
obj = s3.Object('barteks-toxic-comments-stats', 'sample/train_sample100stat_copy.csv')
obj.Acl().put(ACL='public-read')

### Uri

There are two formats of uri:
```
http(s)://s3.amazonaws.com/<bucket>/<object>
http(s)://<bucket>.s3.amazonaws.com/<object>
```

### Example

<https://s3.amazonaws.com/barteks-toxic-comments-stats/sample/train_sample100stat_copy.csv>

## Streaming with smart_open

### Install

```
pip install smart_open
```

In [ ]:
from smart_open import smart_open

comments1000 = \
    pd.read_csv(
        smart_open(
            's3://barteks-toxic-comments/sample/train_sample1000.csv', 'rb', 
            profile_name='myaws'))
    
comments1000_stat =\
    comments1000.groupby(["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"])\
    .count().reset_index()
comments1000_stat.head()

#### Passing session

In [ ]:
pd.read_csv(smart_open(
    's3://barteks-toxic-comments/sample/train_sample100.csv', 'rb', 
        s3_session=session)
).head()

It is smart enough to recognize from where it has to read

In [ ]:
pd.read_csv(smart_open(
    'data/train_sample100.csv', 'rb', 
    s3_session=session)
).head()

#### Writing

In [ ]:
with smart_open('s3://barteks-toxic-comments-stats/sample/train_sample1000stat123.csv', 'w', 
               profile_name='myaws') as fout:
    comments1000_stat.to_csv(fout, index=False)

In [ ]:
import pickle
class Model:

    def __init__(self):
        self.attr = 123
        
model = Model()

with smart_open("s3://barteks-toxic-comments-stats/models/model.pickle", 'wb', 
               profile_name='myaws') as f:
    pickle.dump(model, f, pickle.HIGHEST_PROTOCOL)
    

In [ ]:
list(stat_bucket.objects.all())

In [ ]:
with smart_open("s3://barteks-toxic-comments-stats/models/model.pickle", 'rb', 
               profile_name='myaws') as f:
    model = pickle.load(f)
print(model.attr)

## Links:

* https://github.com/boto/boto3
* https://boto3.amazonaws.com/v1/documentation/api/latest/index.html
* https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge